Sample Programs

In [7]:
import torch
import numpy as np

In [2]:
x = torch.tensor(3.5, requires_grad=True)
y = x*x
z = 2*y + 3
print("x: ",x)
print("y = x*x: ", y)
print("z = 2*y + 3: ", z)

#working out gradients
z.backward()
print("Working out gradients dz/dx")

#gradient at x = 3.5
print("Gradient at x = 3.5: ", x.grad)

x:  tensor(3.5000, requires_grad=True)
y = x*x:  tensor(12.2500, grad_fn=<MulBackward0>)
z = 2*y + 3:  tensor(27.5000, grad_fn=<AddBackward0>)
Working out gradients dz/dx
Gradient at x = 3.5:  tensor(14.)


In [3]:
def f(x):
    return (x-2)**2

def fp(x):
    return 2*(x-2)

x = torch.tensor([1.0],requires_grad=True)
y = f(x)
y.backward()

print('Analytical f\'(x):', fp(x))
print('PyTorch\'s f\'(x):', x.grad)

Analytical f'(x): tensor([-2.], grad_fn=<MulBackward0>)
PyTorch's f'(x): tensor([-2.])


In [4]:
x = torch.tensor([2.0])
x.requires_grad_(True)

y = x**2 + 5
print(y)

tensor([9.], grad_fn=<AddBackward0>)


In [5]:
y.backward() # dy/dx
print('PyTorch gradient: ', x.grad)

with torch.no_grad():
    dy_dx = 2*x

print('Analytical gradient: ', dy_dx)

PyTorch gradient:  tensor([4.])
Analytical gradient:  tensor([4.])


In [8]:
def grad_sigmoid_manual(x):
    """Implements the gradient of the logistic sigmoid function
    #sigma(x) = 1 / (1 + e^{-x})
    """
    # forward pass keeping track of intermediate values for use in the backward pass
    a = -x
    b = np.exp(a)
    c = 1 + b
    s = 1.0/c
    #backward pass
    dsdc = (-1.0/(c**2))
    dsdb = dsdc * 1
    dsda = dsdb * np.exp(a)
    dsdx = dsda * (-1)
    return dsdx

def sigmoid(x):
    y = 1.0 / (1.0 + torch.exp(-x))
    return y

input_x = 2.0
x = torch.tensor(input_x).requires_grad_(True)
y = sigmoid(x)
y.backward()

# Compare the results of manual and automatic gradient functions
print('autograd: ', x.grad.item())
print('manual: ', grad_sigmoid_manual(input_x))

autograd:  0.10499356687068939
manual:  0.1049935854035065


Exercise

In [13]:
input_a = 1.0
input_b = 2.0

a = torch.tensor(input_a).requires_grad_(True)
b = torch.tensor(input_b).requires_grad_(True)

x = 2*a + 3*b
y = 5*a*a + 3*b*b*b
z = 2*x + 3*y

def manually(a,b):
    dzda = 2*2 + 3*5*2*a
    dzdb = 2*3 + 3*3*3*b*b
    return(dzda, dzdb)

print("x = ", x)
print("y = ", y)
print("z = ", z)

z.backward()

print('Gradient dz/da at a = 1 and b = 2: ', a.grad)
print('Gradient dz/db at a = 1 and b = 2: ', b.grad)

print("Manually=: ", manually(input_a, input_b))

x =  tensor(8., grad_fn=<AddBackward0>)
y =  tensor(29., grad_fn=<AddBackward0>)
z =  tensor(103., grad_fn=<AddBackward0>)
Gradient dz/da at a = 1 and b = 2:  tensor(34.)
Gradient dz/db at a = 1 and b = 2:  tensor(114.)
Manually=:  (34.0, 114.0)


In [12]:
# 2
def manual(w, b, x):
    u = w * x
    v = u + b
    a = max(0.0, v)

    if a == 0:
        return 0.0
    else:
        dadv = 1.
        dadu = 1.
        dadb = 1.
        dadw = x
        return dadw, dadb

input_x = 3.0
input_w = 4.0
input_b = 5.0

x = torch.tensor(input_x)
w = torch.tensor(input_w).requires_grad_(True)
b = torch.tensor(input_b).requires_grad_(True)

u = w*x; u.retain_grad()
v = u + b; v.retain_grad()
relu = torch.nn.ReLu();
a = relu(v); a.retain_grad()

a.backward(retain_graph = False)

print(w.grad)
print(b.grad)

grad_w, grad_b = manual(input_w, input_b, input_x)
print(grad_w, grad_b)

tensor(3.)
tensor(1.)
3.0 1.0


In [24]:
# 3

from torch.autograd import grad

def sigmoid_grad_manual(x):
    a = -x
    b = np.exp(a)
    c = 1+b
    e = 1.0/c

    dedc = -1.0/(c**2)
    dedb = dedc*1
    deda = dedb*np.exp(a)
    dedx = deda*(-1)

    return dedx

def manual(w,b,x):
    u = w*x
    v = u + b
    a = sigmoid(v)

    dadv = sigmoid_grad_manual(v)
    dadu = dadv*1
    dadb = dadv*1
    dadw = dadu*x
    return dadw,dadb


inp_x = 5.0
inp_w = 1.0
inp_b = 1.0

x = torch.tensor(inp_x)
w = torch.tensor(inp_w,requires_grad = True)
b = torch.tensor(inp_b,requires_grad = True)

u = w*x
v = u + b
sigmoidal = torch.nn.Sigmoid()
a = sigmoidal(v)

a.backward()

print(w.grad)
print(b.grad)

grad_w,grad_b = manual(inp_w,inp_b,inp_x)
print(grad_w,grad_b)


tensor(0.0123)
tensor(0.0025)
0.012332546456800243 0.0024665092913600485


In [31]:
#4
def f(x):
    return np.exp((-x**2) -2*x - np.sin(x))

def torchF(x):
    return torch.exp((-x**2) - 2*x - torch.sin(x))

def fDiff(x):
    return f(x) * ((-2*x) -2 - np.cos(x))

input_x = 0.0
x = torch.tensor(input_x).requires_grad_(True)
y = torchF(x)

y.backward()

print("PyTorch: ", x.grad)
print("Analytical: ", fDiff(input_x))

PyTorch:  tensor(-3.)
Analytical:  -3.0


In [18]:
#5
def f(x):
    return 8*(x**4) + 3*x**3 + 7*x**2 + 6*x + 3

def fDiff(x):
    return 32*x**3 + 9*x**2 + 14*x + 6

input_x = 2.0
x = torch.tensor(input_x).requires_grad_(True)
y = f(x)

y.backward()

print("PyTorch: ", x.grad)
print("Analytical: ", fDiff(input_x))

PyTorch:  tensor(326.)
Analytical:  326.0


In [35]:
# 6

def manual(x, y, z):
    a = 2 ** x
    b = np.sin(y)
    c = a / b
    d = c * z
    e = np.log(d + 1)
    f = np.tanh(e)
    print("Forward Pass: ")
    print("a: ", a)
    print("b: ", b)
    print("c: ", c)
    print("d: ", d)
    print("e: ", e)
    print("f: ", f)

    df_de = (1 - np.tanh(e)**2)
    de_dd = 1 / (d+1)
    dd_dc = z
    dc_da = 1 / b
    da_dx = 2
    dc_db = -a / b**2
    db_dy = np.cos(y)
    print("df_de: ", df_de)
    print("de_dd: ", de_dd)
    print("dd_dc: ", dd_dc)
    print("dc_db: ", dc_db)
    print("dc_da: ", dc_da)
    print("da_dx: ", da_dx)
    print("db_dy: ", db_dy)

    yy = df_de * de_dd * dd_dc * dc_db * db_dy
    xx = df_de * de_dd * dd_dc * dc_da * da_dx
    print("xx: ", xx)
    print("yy: ", yy)
    return (yy, xx)

input_x = 2.0
input_y = 3.0
input_z = 4.0

x = torch.tensor(input_x).requires_grad_(True)
y = torch.tensor(input_y).requires_grad_(True)
z = torch.tensor(input_z).requires_grad_(True)

a = 2 ** x; a.retain_grad()
b = torch.sin(y); b.retain_grad()
c = a / b; c.retain_grad()
d = c * z; d.retain_grad()
e = torch.log(d + 1); e.retain_grad()
f = torch.tanh(e)

f.backward()

print("PyTorch: ", x.grad)
print("PyTorch: ", y.grad)
print("PyTorch: ", z.grad)
print("e: ",e.grad)
print("d: ",d.grad)
print("c: ",c.grad)
print("b: ",b.grad)
print("a: ",a.grad)
manual(input_x, input_y, input_z)

PyTorch:  tensor(0.0002)
PyTorch:  tensor(0.0021)
PyTorch:  tensor(7.5769e-05)
e:  tensor(0.0003)
d:  tensor(2.6731e-06)
c:  tensor(1.0692e-05)
b:  tensor(-0.0021)
a:  tensor(7.5769e-05)
Forward Pass: 
a:  4.0
b:  0.1411200080598672
c:  28.344669582948747
d:  113.37867833179499
e:  4.739514683356544
f:  0.9998471354956351
df_de:  0.00030570564117315957
de_dd:  0.008742888225191356
dd_dc:  4.0
dc_db:  -200.85507344163494
dc_da:  7.086167395737187
da_dx:  2
db_dy:  -0.9899924966004454
xx:  0.00015151644546128608
yy:  0.002125852261213157


(0.002125852261213157, 0.00015151644546128608)